In [ ]:
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import panel as pn 
import copy
from collections import Counter
import csv
from sklearn.metrics import confusion_matrix
from scipy import stats
from statsmodels.distributions.empirical_distribution import ECDF
from bokeh.io import output_notebook, export_png
from bokeh.plotting import figure,show 
from bokeh.layouts import column, row, grid
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
output_notebook()
pn.extension()

In [ ]:
def get_config():
    with open("config.yaml", 'r') as stream:
        config = yaml.safe_load(stream)
    return config

In [ ]:
def combine_datasets(ccre, euclidean):
    config = get_config()

    ccre_df = pd.read_csv(config[ccre])
  
    euclidean_df = pd.read_csv(config[euclidean])
    
    #combine
    df = pd.concat([euclidean_df, ccre_df], axis=1)
    df = df.loc[:,~df.columns.duplicated()].copy()
    hospitals = df['hospital'].str.split('_') 

    same_centre_bools = [] 
    for hospital in hospitals:
        if hospital[0] == hospital[1]:
            same_centre_bools.append(True)
        else:
            same_centre_bools.append(False)
    df['same_centre'] = same_centre_bools
    return df

In [ ]:
df = combine_datasets(ccre="ccre_smote", euclidean="euclidean_smote")
df.loc[df['ccre(X|Y)'].str[0] == '[', "ccre(X|Y)"] = 0
df = df.astype({'ccre(X|Y)':'float'})
df = df.astype({'euclidean_similarity':'float'})
# patients_to_remove = [23, 25, 27,30,32,34,35,36,40,52, 56, 104,110,111,117,120,134,140,148,159,161,162, 171,179,180,181,182,191,192,196,205,211,215,216, 230,233,236,237,238,239,242,243,245, 247,252,253,254,256,258,262,272,275,277,279,284,285,286,287,289,290,291,292,294,295,296,297,298,299,300,301,303]
# df = df[(~df['id_x'].isin(patients_to_remove)) & ~df['id_y'].isin(patients_to_remove)]

In [ ]:
df.fillna({'ccre(X|Y)': 1}, inplace=True)
df

In [ ]:
def compare_scatter_plots(disease1="HC", disease2="HC"):
    combine_diseases = f"{disease1}_{disease2}"
    figures = []
    for i in sources_names:
        for j in sources_names:

            combine_centres = f"{i}_{j}"
            dataset = df[(df.hospital == combine_centres) & (df.diagnosis == combine_diseases)]
         

            figure1 = figure(height=400, width=400, title=f"{combine_centres} {combine_diseases}")
            figure1.scatter('ccre(X|Y)', 'euclidean_similarity', source=dataset)
            figure1.xaxis.axis_label = "ccre"
            figure1.yaxis.axis_label = "euclidean similarity"
            figures.append(figure1)
    
    return grid([
        [figures[0], figures[1], figures[2]],
        [figures[3], figures[4], figures[5]],
        [figures[6], figures[7], figures[8]]
    ])
            

sources_names = ["UMCG", "UGOSM", "CUN"]
diagnosis = ["HC", "AD", "PD"]
scatter_plots = pn.interact(compare_scatter_plots, disease1=diagnosis, disease2=diagnosis)
scatter_plots

In [ ]:
def cumulative_histogram(diagnosis1 = "PD", diagnosis2 = "PD"):
    
    combine_diseases = f"{diagnosis1}_{diagnosis2}"
    figures = []
    for i in sources_names:

        for j in sources_names:

            combine_centres = f"{i}_{j}"
            data = df[(df.hospital == combine_centres) & (df.diagnosis == combine_diseases)]
            
            ecdf = ECDF(data['ccre(X|Y)'])
            ecdf2 = ECDF(data['euclidean_similarity'])
           
            plot1 = figure(width=400, height=400, title=f"{combine_centres} {combine_diseases}")
            plot1.line(ecdf.x, ecdf.y)
            plot1.line(ecdf2.x, ecdf2.y, color='orange')
          
            figures.append(plot1)
    

    return grid([
        [figures[0], figures[1], figures[2]],
        [figures[3], figures[4], figures[5]],
        [figures[6], figures[7], figures[8]]
    ])
cumulative_plots = pn.interact(cumulative_histogram, diagnosis1=diagnosis, diagnosis2=diagnosis)
cumulative_plots


In [ ]:
sorted_df = df.sort_values(['hospital', 'diagnosis'])
sorted_df

In [ ]:
list_of_id = sorted_df['id_x'].unique()

def adjacency_matrix(metric):
    squared_matrix = []
    temp_values = []
    for i in list_of_id:
 
        

        for j in list_of_id:
         
            if i == j:
                temp_values.append(1)
            else:
                
                temp_values.append(sorted_df[(sorted_df['id_x'] == i) & (sorted_df['id_y'] == j)][metric].values[0])
        if len(temp_values) == len(list_of_id):
            
            squared_matrix.append(temp_values)
            temp_values = []
    return squared_matrix
CCRE_matrix = adjacency_matrix('ccre(X|Y)')
euclidean_matrix = adjacency_matrix('euclidean_similarity')
euclidean_distance_matrix = adjacency_matrix('euclidean_distance')


In [ ]:

hospital_diagnosis_full = []

for i in list_of_id:
    
    hospital = df[df['id_x'] == i]['hospital'].values[0].split('_')[0]
    diagnosis = df[df['id_x'] == i]['diagnosis'].values[0].split('_')[0]
    hospital_diagnosis_full.append(f'{hospital}_{diagnosis}')


In [ ]:
position = [16, 49, 91, 135, 183, 201, 220, 232]
hospital_diagnosis = ['CUN_HC', 'CUN_PD', 'UGOSM_AD', 'UGOSM_HC', 'UGOSM_PD', 'UMCG_AD', 'UMCG_HC', 'UMCG_PD']
plt.xticks(position, hospital_diagnosis, rotation='vertical')
plt.yticks(position, hospital_diagnosis)
plt.title('HEAT map ccre adjacency matrix')
plt.imshow(CCRE_matrix, vmin=0, vmax=1)


In [ ]:

plt.xticks(position, hospital_diagnosis, rotation='vertical')
plt.yticks(position, hospital_diagnosis)
plt.title('HEAT map euclidean similarity adjacency matrix')
plt.imshow(euclidean_matrix, vmin=0, vmax=0.1)

In [ ]:
ill_notill = []
centre_ill_notill = []
one_or_zero = []
for i in hospital_diagnosis_full:
    centre = i.split('_')[0]
    if i[-2:] == 'AD' or i[-2:] ==  'PD':
        ill_notill.append(f'ill')
        centre_ill_notill.append(f'{centre}_ill')
        one_or_zero.append(1)
    else:
        ill_notill.append('notill')
        centre_ill_notill.append(f'{centre}_notill')
        one_or_zero.append(0)


In [ ]:
HC_PD_AD = []
for i in hospital_diagnosis_full:
    if i[-2:] == 'PD':
        HC_PD_AD.append('PD')
    elif i[-2:] == 'AD':
        HC_PD_AD.append('AD')
    elif i[-2:] == 'HC':
        HC_PD_AD.append('HC')


<h1>LDA</h1>

In [ ]:
def lda(X, y, title):
     #devide the data
    ill = []
    notill = []
    for index, label in enumerate(y):
        if label == 0:
            notill.append(X[index])
        else:
            ill.append(X[index])
    ill = np.array(ill)
    notill = np.array(notill)
   
    # calculate mean and covariance
    Mill = ill.mean(axis=0)
    Mnotill = notill.mean(axis=0)
    nill = ill.shape[0]
    nnotill = notill.shape[0]
    Cill = np.cov(ill.T)
    Cnotill = np.cov(notill.T)

    #calculate the pooled covariance matrix
    Sp = ((nnotill - 1) * Cnotill + (nill - 1) * Cill) / (nnotill + nill - 2)

    #project data onto the discriminant axis
    a = np.linalg.inv(Sp) @ (Mill - Mnotill) 
    a /= np.sqrt((a ** 2).sum())

    tw = ill @ a 
    tr = notill @ a

    #plot 
    Ew = ill - tw[:, None] * a[None, :]
    Er = notill - tr[:, None] * a[None, :]

    X = np.concatenate((Ew, Er), axis=0)
    S = np.cov(X.T)
    vals, vecs = np.linalg.eig(S)
    order = vals.argsort()[::-1]
    vecs = vecs[:, order]
    vals = vals[order]
    Tw = ill @ vecs
    Tr = notill @ vecs
    plt.title(f"LDA {title}")
    plt.scatter(tr, Tr[:, 0], s=5, c="red", label='notill')
    plt.scatter(tw, Tw[:, 0], s=5, c="orange", label='ill')
    plt.xlabel("Discriminant axis")
    plt.ylabel("Non-discriminant PC 1")
    plt.legend()

lda(CCRE_matrix, one_or_zero, "CCRE")

In [ ]:

plt.ylim(-0.7,0)
lda(euclidean_matrix, one_or_zero, "euclidean")

PCA

In [ ]:
def PCA_visualization(X, label_list, components, colors, title):
    pca = PCA(n_components=components)
    pca_fit = pca.fit_transform(X)
    data = {'pc1': pca_fit[:, 0], 'pc2':pca_fit[:, 1], 'labels': label_list}
    df = pd.DataFrame(data).values
    for i in df:
        plt.scatter(i[0], i[1], color=colors[i[2]],label=i[2])
    handles, labels = plt.gca().get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    plt.title(title)
    plt.xlabel('first PC')
    plt.ylabel('second PC')
    plt.legend(by_label.values(), by_label.keys(), loc='center left', bbox_to_anchor=(1, 0.5))



In [ ]:
X = np.array(CCRE_matrix)
colors = {'UMCG_HC': 'black', 'UMCG_PD': 'dimgrey', 'UMCG_AD': 'silver', 'CUN_HC': 'saddlebrown', 'CUN_PD': 'sienna', 'UGOSM_AD': 'purple', 'UGOSM_HC': 'violet', 'UGOSM_PD': 'fuchsia'}
PCA_visualization(X, hospital_diagnosis_full, len(X), colors, 'PCA ccre similarity')

In [ ]:
X_eu = np.array(euclidean_matrix)
PCA_visualization(X_eu, hospital_diagnosis_full, len(X), colors, 'PCA euclidean similarity')

In [ ]:
X_eu_distance = np.array(euclidean_distance_matrix)
PCA_visualization(X_eu_distance, hospital_diagnosis_full, len(X), colors, 'PCA euclidean distance')

In [ ]:
colors = {'ill': 'maroon', 'notill': 'lightcoral'}
PCA_visualization(X, ill_notill, len(X), colors, 'PCA ccre similarity')


In [ ]:
PCA_visualization(X_eu, ill_notill, len(X), colors, 'PCA euclidean similarity')

In [ ]:
PCA_visualization(X_eu_distance, ill_notill, len(X), colors, 'PCA euclidean distance')

In [ ]:
colors = {'CUN_notill': 'black', 'CUN_ill': 'dimgrey', 'UGOSM_ill': 'purple', 'UGOSM_notill': 'violet', 'UMCG_ill': 'navy', 'UMCG_notill': 'mediumblue'}
PCA_visualization(X, centre_ill_notill, len(X), colors, 'PCA ccre similarity')

In [ ]:
PCA_visualization(X_eu, centre_ill_notill, len(X), colors, 'PCA euclidean similarity')

In [ ]:
PCA_visualization(X_eu_distance, centre_ill_notill, len(X), colors, 'PCA euclidean distance')

In [ ]:
colors = {'AD': 'maroon', 'PD': 'lightcoral', 'HC': 'green'}
PCA_visualization(X, HC_PD_AD, len(X), colors, 'PCA ccre similarity')

In [ ]:
PCA_visualization(X_eu, HC_PD_AD, len(X), colors, 'PCA euclidean similarity')

In [ ]:
PCA_visualization(X_eu_distance, HC_PD_AD, len(X), colors, 'PCA euclidean distance')

In [ ]:
def compare_metrics(hospital, first_diagnosis, second_diagnosis):

    first = df[(df['hospital'] == hospital) & (df['diagnosis'] == f"{first_diagnosis}_{first_diagnosis}")]
    second = df[(df['hospital'] == hospital) & (df['diagnosis'] == f"{second_diagnosis}_{second_diagnosis}")]

    third = df[(df['hospital'] == hospital) & (df['diagnosis'] == f"{first_diagnosis}_{second_diagnosis}")]
    fourth = df[(df['hospital'] == hospital) & (df['diagnosis'] == f"{second_diagnosis}_{first_diagnosis}")]
    
    #euclidean distance
    euclidean_first = first['euclidean_distance'].median() / (pd.concat([third['euclidean_distance'], fourth['euclidean_distance']])).median()
    euclidean_second = second['euclidean_distance'].median() / (pd.concat([third['euclidean_distance'], fourth['euclidean_distance']])).median()
    
    #ccre
    ccre_first = (1- first['ccre(X|Y)']).median() / (pd.concat([1 - third['ccre(X|Y)'], 1 - fourth['ccre(X|Y)']])).median()
    ccre_second = (1-second['ccre(X|Y)']).median() / (pd.concat([1 - third['ccre(X|Y)'], 1 - fourth['ccre(X|Y)']])).median()
    
    header = ['formula', 'hospital','first_diagnosis', 'second_diagnosis' 'ccre_average', 'euclidean distance average', 'ccre<euclidean']
    data_first_formula = [
        f'mean({first_diagnosis})/ mean(between {first_diagnosis} and {second_diagnosis})',
        hospital,
        first_diagnosis,
        second_diagnosis,
        ccre_first,
        euclidean_first,
        ccre_first<euclidean_first
    ]
    data_second_formula = [
        f'mean({second_diagnosis})/ mean(between {first_diagnosis} and {second_diagnosis})',
        hospital,
        first_diagnosis,
        second_diagnosis,
        ccre_second,
        euclidean_second,
        f'{ccre_second<euclidean_second}'
    ]
    file_created = False
    with open(f"compare_euclidean_ccre_distance.csv", "a", newline="") as file:
        writer = csv.writer(file)
        
        if file_created:
            
            writer.writerow(header)
            writer.writerow(data_first_formula)
            writer.writerow(data_second_formula)
            file_created = True
        else:
            writer.writerow(data_first_formula)
            writer.writerow(data_second_formula)
    print(f'formula 1 mean({first_diagnosis})/ mean(between {first_diagnosis} and {second_diagnosis}) , ccre average = {ccre_first}, euclidean distance average = {euclidean_first}, ccre < euclidean similarity {ccre_first<euclidean_first}')
    print(f'formula 2 mean({second_diagnosis})/ mean(between {first_diagnosis} and {second_diagnosis}) , ccre average = {ccre_second}, euclidean distance average = {euclidean_second}, ccre < euclidean similarity {ccre_second<euclidean_second}\n')


In [ ]:
hospitals = ['UMCG_UMCG', 'UMCG_UGOSM', 'UMCG_CUN', 'UGOSM_UMCG', 'UGOSM_UGOSM',
       'UGOSM_CUN', 'CUN_UMCG', 'CUN_UGOSM', 'CUN_CUN']
for hospital in hospitals:
    print(hospital)
    compare_metrics(hospital, 'HC', 'PD')
    if "CUN" not in hospital:
       compare_metrics(hospital, 'HC', 'AD')
       compare_metrics(hospital, 'AD', 'PD')


In [ ]:
config = get_config()

compare_euclidean_ccre_df = pd.read_csv(config['compare_euclidean_ccre'])
compare_within = compare_euclidean_ccre_df[compare_euclidean_ccre_df['hospital'].isin(['UMCG_UMCG', 'CUN_CUN', 'UGOSM_UGOSM'])]
compare_outside = compare_euclidean_ccre_df[~compare_euclidean_ccre_df['hospital'].isin(['UMCG_UMCG', 'CUN_CUN', 'UGOSM_UGOSM'])]
compare_euclidean_ccre_df

<h1>HC == HC</h1>

In [ ]:

def plot_pie(values, title):
 
    counter = Counter(values)
    values = [counter[0]/values.count()* 100, counter[1]/values.count()*100]
    labels = ['False', 'True']
    plt.title(title)
    plt.pie(values, labels=labels)
    plt.legend()


In [ ]:
values = compare_within[compare_within['formula'].str[5:7] == 'HC']['ccre<euclidean'].astype(int)

plot_pie(values, 'within centres HC ccre<euclidean median')
compare_within[compare_within['formula'].str[5:7] == 'HC']


In [ ]:
values = compare_outside[compare_outside['formula'].str[5:7] == 'HC']['ccre<euclidean'].astype(int)
plot_pie(values, 'between centres HC ccre<euclidean median')

<h1>HC != PD or AD</h1>

In [ ]:
compare_euclidean_ccre_df[(compare_euclidean_ccre_df['formula'].str[5:7] == 'HC') &(compare_euclidean_ccre_df['second_diagnosis'].isin(['PD', 'AD']))]

<h1>AD or PD != AD or PD</h1>

In [ ]:
values = compare_within[compare_within['first_diagnosis'] == 'AD']['ccre<euclidean'].astype(int)
plot_pie(values, 'within PD or AD != PD or AD ccre<euclidean median')

In [ ]:
values = compare_outside[compare_outside['first_diagnosis'] == 'AD']['ccre<euclidean'].astype(int)
plot_pie(values, 'between centres PD or AD != PD or AD ccre<euclidean median')
compare_outside[compare_outside['first_diagnosis'] == 'AD']

<h1>PD > AD</h1>

In [ ]:
compare_euclidean_ccre_df[compare_euclidean_ccre_df['formula'].str[5:7].isin(['PD', 'AD'])]

<h1>kNN</h1>
Because we have all the distances we dont need to implement the actual model but instead can use the adjacency matrix

In [ ]:
def create_labels(sorted_df, list_of_test_ids):
    list_of_labels = []
    for i in list_of_id:
        hospital = sorted_df[sorted_df['id_x'] == i].iloc[0]['hospital'].split('_')[0]
        diagnosis = sorted_df[sorted_df['id_x'] == i].iloc[0]['diagnosis'].split('_')[0]
        if sorted_df[sorted_df['id_x'] == i].iloc[0]['id_x'] in list_of_test_ids:
            list_of_labels.append(f'{hospital}_{diagnosis}_test')
        else:
            list_of_labels.append(f'{hospital}_{diagnosis}')
    return list_of_labels


In [ ]:
def combine_label_with_id(list_of_labels):
    oversampled_data = [[label,index] for index, label in enumerate(list_of_labels)]
    return oversampled_data


In [ ]:
def split_data(data):
 
    test_set = []
    for data_point in data:
       
        if 'test' in data_point[0]:

            test_set.append(data_point)
    
    training_set = [i for i in data if (str(i[1]) not in np.array(test_set)[:, 1])]
    
    return test_set, training_set


In [ ]:
def knn_3_classes(k, test_X, test_y, original_matrix, training_set):
    correct_predicted = []
    for index, row in enumerate(np.array(test_y)[:, 1]):
        
        actual = test_y[index]
        predicted = []
        for K in range(k):
            prediction_number=[i for i in range(len(original_matrix[int(row)])) if original_matrix[int(row)][i] == sorted(test_X[index], reverse=True)[K]][0]
            label = [i for i in training_set if i[1] == prediction_number][0][0].split('_')[1]
            predicted.append(label)
        
        
        counter = Counter(predicted)
        predicted_value = max(counter, key=counter.get)
        actual = actual[0].split('_')[1]
        
        correct_predicted.append([predicted_value, actual, predicted_value == actual])
    return correct_predicted

In [ ]:
list_of_labels = create_labels(sorted_df, np.arange(304,351))
oversampled_data = combine_label_with_id(list_of_labels)
test_set, training_set = split_data(oversampled_data)

CCRE_training = [[CCRE_matrix[int(i)][int(j)] for j in np.array(training_set)[:, 1]] for i in np.array(training_set)[:, 1]]
CCRE_test = [[CCRE_matrix[int(i)][int(j)] for j in np.array(training_set)[:, 1]] for i in np.array(test_set)[:, 1]]
euclidean_training = [[euclidean_matrix[int(i)][int(j)] for j in np.array(training_set)[:, 1]] for i in np.array(training_set)[:, 1]]
euclidean_test = [[euclidean_matrix[int(i)][int(j)] for j in np.array(training_set)[:, 1]] for i in np.array(test_set)[:, 1]]
def execute_knn(name, metric_test, test_set, metric_matrix, training_set):
    print(name)          
    for i in range(1, 9):
        test_ccre = knn_3_classes(i, metric_test, test_set, metric_matrix, training_set)
        predicted = np.array(test_ccre)[:, 0]
        actual = np.array(test_ccre)[:, 1]
        
        print(confusion_matrix(actual, predicted, labels=["HC", "PD", "AD"]))
        
        true = 0
        false = 0
        for j in test_ccre:
            if j[2]:
                true +=1
            else:
                false +=1

        print(f'neighbours = {i}, incorrect predicted = {false}, correct predicted = {true}')
execute_knn('CCRE', CCRE_test, test_set, CCRE_matrix, training_set)


In [ ]:
execute_knn('CCRE', euclidean_test, test_set, euclidean_matrix, training_set)